In [56]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# 1. Generate synthetic dataset
np.random.seed(42)
n = 1000

x = np.random.uniform(0, 2*np.pi, n)

x1 = x**3
x2 = np.sin(x)

mu = 1
sigma2 = 0.2
noise = np.random.normal(mu, np.sqrt(sigma2), n)

y = 2 - x1 + 3*x2 + noise

# Metrics
def regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    ndei = rmse / np.std(y_true)
    return mse, rmse, ndei

In [57]:
# (الف) Model 1: feature = x^2 , b = 2 (fixed)

X = x**2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ---- without normalization ----
y_train_adj = y_train - 2
w_hat = np.sum(X_train * y_train_adj) / np.sum(X_train**2)

y_pred_train = 2 + w_hat * X_train
y_pred_test = 2 + w_hat * X_test

m1_train = regression_metrics(y_train, y_pred_train)
m1_test = regression_metrics(y_test, y_pred_test)
print("m1_train: Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m1_train))
print("m1_test:  Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m1_test))


m1_train: Total Error = 263.50, RMSE = 16.23, R2 = 0.224
m1_test:  Total Error = 258.30, RMSE = 16.07, R2 = 0.237


In [58]:
# ---- with normalization ----
scaler = StandardScaler()
X_train_n = scaler.fit_transform(X_train.reshape(-1,1)).flatten()
X_test_n = scaler.transform(X_test.reshape(-1,1)).flatten()

y_train_adj = y_train - 2
w_hat_n = np.sum(X_train_n * y_train_adj) / np.sum(X_train_n**2)

y_pred_train_n = 2 + w_hat_n * X_train_n
y_pred_test_n = 2 + w_hat_n * X_test_n

m1_train_n = regression_metrics(y_train, y_pred_train_n)
m1_test_n = regression_metrics(y_test, y_pred_test_n)
print("m1_train: Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m1_train_n))
print("m1_test:  Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m1_test_n))


m1_train: Total Error = 3816.77, RMSE = 61.78, R2 = 0.851
m1_test:  Total Error = 3783.53, RMSE = 61.51, R2 = 0.906


In [59]:
# (ب) Model 2: x1 & x2 , NO bias
X12 = np.column_stack((x1, x2))

X_train, X_test, y_train, y_test = train_test_split(
    X12, y, test_size=0.2, random_state=42
)

lr_no_bias = LinearRegression(fit_intercept=False)
lr_no_bias.fit(X_train, y_train)

y_pred_train = lr_no_bias.predict(X_train)
y_pred_test = lr_no_bias.predict(X_test)

m2_train = regression_metrics(y_train, y_pred_train)
m2_test = regression_metrics(y_test, y_pred_test)

print("m1_train: Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m2_train))
print("m1_test:  Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m2_test))


m1_train: Total Error = 4.06, RMSE = 2.02, R2 = 0.028
m1_test:  Total Error = 4.22, RMSE = 2.06, R2 = 0.030


In [60]:
# (ج) Model 3: x1 & x2 WITH bias
lr_bias = LinearRegression(fit_intercept=True)
lr_bias.fit(X_train, y_train)

y_pred_train = lr_bias.predict(X_train)
y_pred_test = lr_bias.predict(X_test)

m3_train = regression_metrics(y_train, y_pred_train)
m3_test = regression_metrics(y_test, y_pred_test)

print("m3_train: Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m3_train))
print("m3_test:  Total Error = {:.2f}, RMSE = {:.2f}, R2 = {:.3f}".format(*m3_test))
#Total Error ≈ Bias² + Variance + Noise

m3_train: Total Error = 0.20, RMSE = 0.45, R2 = 0.006
m3_test:  Total Error = 0.17, RMSE = 0.41, R2 = 0.006


In [62]:
# (د) Effect of noise parameters

def noise_experiment(mu, sigma2):
    noise = np.random.normal(mu, np.sqrt(sigma2), n)
    y_new = 2 - x1 + 3*x2 + noise

    X_tr, X_te, y_tr, y_te = train_test_split(
        X12, y_new, test_size=0.2, random_state=42
    )

    model = LinearRegression()
    model.fit(X_tr, y_tr)

    y_pred = model.predict(X_te)
    return regression_metrics(y_te, y_pred)

noise_results = {
    "low_noise": noise_experiment(0, 0.05),
    "base_noise": noise_experiment(1, 0.2),
    "high_noise": noise_experiment(2, 0.5)
}


for key, (total_error, rmse, r2) in noise_results.items():
    print(f"{key}: Total Error = {total_error:.3f}, RMSE = {rmse:.3f}, R² = {r2:.3f}")


low_noise: Total Error = 0.048, RMSE = 0.220, R² = 0.003
base_noise: Total Error = 0.211, RMSE = 0.459, R² = 0.007
high_noise: Total Error = 0.520, RMSE = 0.721, R² = 0.011


1️⃣ Bias کم، خطا کم

مدل ساده یا پیچیده ولی خوب fit کرده.

هم روی train و هم روی test خطا کم هست.

وضعیت: مدل عالی، نه underfit و نه overfit.

مثال: Linear Regression خوب روی داده‌های خطی.

2️⃣ Bias کم، خطا زیاد

مدل پیچیده که روی train خیلی خوبه ولی روی test ضعیفه.

Overfitting: مدل جزئیات نویز train رو هم یاد گرفته.

R² روی train بالا، روی test پایین.

راه حل: Regularization (Ridge/Lasso)، کاهش پیچیدگی مدل، افزایش داده.

3️⃣ Bias زیاد، خطا کم

این حالت خیلی نادره، معمولاً وقتی داده‌ها خیلی نزدیک صفر یا بدون تغییر هستند اتفاق میفته.

مدل ساده است، خطا عددی کم چون مقادیر هدف کوچک هستند، ولی مدل هیچ رابطه‌ای یاد نگرفته → R² پایین.

مثال: مدل همیشه میانگین y رو پیش‌بینی می‌کنه.

وضعیت: underfitting ولی خطا ظاهراً کم است.

4️⃣ Bias زیاد، خطا زیاد

مدل خیلی ساده و ضعیف برای داده‌های پیچیده.

نه روی train و نه روی test خوب کار نمی‌کنه.

Underfitting شدید: مدل نمی‌تونه رابطه‌ها رو یاد بگیره.

مثال: Linear Regression ساده روی داده‌های غیرخطی شدید.